# 1. Initialization

### i) Configure RPCs & deployer

In [ ]:
from dotenv import load_dotenv
import os
import boa
from eth_account import Account
from web3 import Web3
import logging
import subprocess
import time

from ABIs import createX_abi, lzreadlib_abi

# logging.basicConfig(filename="deploy.log", filemode='a', level=logging.INFO, format="%(asctime)s -  %(levelname)s - %(message)s")
logging.basicConfig(level=logging.INFO, format="%(asctime)s -  %(levelname)s - %(message)s")

load_dotenv()
main_chain = "sepolia"

PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")

RPCs = {
    "sepolia": "https://eth-sepolia.public.blastapi.io",
    "base-sepolia": "https://sepolia.base.org",
    "optimism-sepolia": "https://sepolia.optimism.io",
    # "arbitrum-sepolia": "https://sepolia-rollup.arbitrum.io/rpc",
}

createX_address = "0xba5Ed099633D3B313e4D5F7bdc1305d3c28ba5Ed"

state_dict = {}

deployer = Account.from_key(PRIVATE_KEY)

for key in RPCs.keys():
    state_dict[key] = {}
    state_dict[key]["rpc"] = RPCs[key]
    boa.set_network_env(RPCs[key])
    boa.env.add_account(deployer)
    state_dict[key]["boa"] = boa.env  # oops
    state_dict[key]["w3"] = Web3(Web3.HTTPProvider(RPCs[key]))
    state_dict[key]["createx"] = boa.loads_abi(createX_abi).at(createX_address)

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        logging.info(f"Working with {boa.env.eoa} on {key}, id {boa.env.evm.patch.chain_id}")
        logging.info(
            f"Chain balance is {state_dict[key]['w3'].eth.get_balance(boa.env.eoa)/1e18 :.3f} ETH"
        )
        logging.info(f"CreateX test: {state_dict[key]['createx'].computeCreate2Address(b'',b'')}")

### ii) Parse LZ deployments data (libs and endpoints)

In [ ]:
from LZDeployments import LZDeployments

lz = LZDeployments()
for chain in state_dict:
    metadata = lz.get_chain_metadata(chain)["metadata"]
    dvn_data = lz.get_chain_metadata(chain)

    # Update state dict
    state_dict[chain].update(
        {
            "eid": metadata["eid"],
            "endpoint": metadata["endpointV2"],
            "send_lib": metadata.get("sendUln302", "unavailable"),
            "receive_lib": metadata.get("receiveUln302", "unavailable"),
            "read_lib": metadata.get("readLib1002", "unavailable"),
            "dvns": dvn_data["dvns"],
        }
    )

    # Print info
    logging.info(f"LZ details for {chain}:")
    logging.info(f"Chain eID: {metadata['eid']}\nEndpoint address: {metadata['endpointV2']}")
    logging.info(f"DVNs: {len(dvn_data['dvns'])}, Read DVNs: {len(dvn_data['dvns_lzread'])}")
    logging.info(
        f"Send lib: {state_dict[chain]['send_lib']}\n"
        f"Receive lib: {state_dict[chain]['receive_lib']}\n"
        f"Read lib: {state_dict[chain]['read_lib']}\n---"
    )

# 2. Contracts deployment

### i) Deploy main view contract and block oracles 

In [ ]:
# On mainnet viewer will tell us ground truth block data
# On other chains block oracles are consensus of various messengers (LZ in this script)
random_bytes = os.urandom(11)
for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        if key == main_chain:
            # deploying view contract on mainnet
            contract_deployer = boa.load_partial("../contracts/MainnetBlockView.vy")
            contract = contract_deployer()
            logging.info(f"Block view deployed at {contract.address} on {key}")
            state_dict[key]["oracle"] = contract
        else:
            # deploying block oracle on other chains
            contract_deployer = boa.load_partial("../contracts/BlockOracle.vy")
            bytecode = contract_deployer.compiler_data.bytecode
            args = boa.util.abi.abi_encode("(uint256,address)", (1, boa.env.eoa))
            deploycode = bytecode + args
            guard_bytes = bytes.fromhex(boa.env.eoa[2:] + "00")
            salt = guard_bytes + random_bytes
            address = state_dict[key]["createx"].deployCreate2(salt, deploycode)
            # contract = contract_deployer(1)  # commit threshold
            logging.info(f"Block oracle deployed at {address} on {key}")
            state_dict[key]["oracle"] = contract_deployer.at(address)

### ii) Deploy block relay contracts

In [ ]:
# Deploy LZBlockRelay on each chain
# This contract is used to send and receive messages via LZ, including LZRead and chained broadcasting
random_bytes = os.urandom(11)
for key in state_dict:
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        contract_deployer = boa.load_partial("../contracts/messengers/LZBlockRelay.vy")
        bytecode = contract_deployer.compiler_data.bytecode
        args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
        deploycode = bytecode + args
        guard_bytes = bytes.fromhex(boa.env.eoa[2:] + "00")
        salt = guard_bytes + random_bytes
        address = state_dict[key]["createx"].deployCreate2(salt, deploycode)
        contract = contract_deployer.at(address)
        # contract = contract_deployer(
        #     state_dict[key]["endpoint"],  # endpoint on the chain
        #     300_000,  # default gas limit
        #     read_channel,  # read channel
        # )
        print(f"LZ Messenger deployed at {contract.address} on {key}")
        state_dict[key]["block_relay"] = contract

# 3. Contracts configuration

## i) Oracles configuration

In [ ]:
for key in state_dict.keys():
    if key == main_chain:
        continue
    relay_contract = state_dict[key]["block_relay"]

    # initialize block relay
    read_channel = 4294967295 if not state_dict[key]["read_lib"] == "unavailable" else 0
    endpoint = state_dict[key]["endpoint"]
    default_gas_limit = 300_000
    relay_contract.initialize(
        state_dict[key]["endpoint"],  # endpoint on the chain
        default_gas_limit,  # default gas limit
        read_channel,  # read channel
        [],
        [],  # empty arrays at init (can optimzie and pass many)
    )

    # add block oracles everywhere
    if relay_contract.block_oracle() == state_dict[key]["oracle"].address:
        logging.info(f"Skipping {key} - already set")
    else:
        relay_contract.set_block_oracle(state_dict[key]["oracle"].address)
        logging.info(f"Set block oracle tx on {key}")

    # now add relay as committer to block oracle
    oracle_contract = state_dict[key]["oracle"]
    if oracle_contract.is_committer(relay_contract.address):
        logging.info(f"Skipping {key} - already a committer")
    else:
        oracle_contract.add_committer(relay_contract.address)
        logging.info(f"Add committer tx on {key}")

    # add mainnet view contract on read chains (as read source)
    if state_dict[key]["read_lib"] != "unavailable":
        contract = state_dict[key]["block_relay"]
        contract.set_read_config(
            True, state_dict[main_chain]["eid"], state_dict[main_chain]["oracle"].address
        )
        logging.info(f"Set read config on {key}")

## ii) LZ Configuration

### a) Send/receive libs (uln302)

In [ ]:
# send lib
for key in state_dict.keys():
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        # send lib
        state_dict[key]["block_relay"].set_lz_send_lib(
            state_dict[key]["eid"], state_dict[key]["send_lib"]
        )
        logging.info(f"Added send lib: on {key}")
        # receive lib
        state_dict[key]["block_relay"].set_lz_receive_lib(
            state_dict[key]["eid"], state_dict[key]["receive_lib"]
        )
        logging.info(f"Added receive lib: on {key}")

### b) Read libs

In [ ]:
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        w3 = state_dict[key]["w3"]
        lzreadlib = w3.eth.contract(address=state_dict[key]["read_lib"], abi=lzreadlib_abi)
        logging.info(
            f"Read lib type: {lzreadlib.functions.messageLibType().call()}, version: {lzreadlib.functions.version().call()}"
        )
        eid_check = [4294967295, 4294967294]
        supported_eid = None
        for eid in eid_check:
            support = lzreadlib.functions.isSupportedEid(eid).call()
            logging.info(f"Supports {eid}: {support}")
            if support and not supported_eid:
                supported_eid = eid
                break
        # make sure read channel is supported
        read_channel = state_dict[key]["block_relay"].LZ_READ_CHANNEL()
        logging.info(
            f"Relayer's {state_dict[key]['block_relay'].address} read channel on {key}: {read_channel}"
        )
        if supported_eid and read_channel != supported_eid:
            logging.info(f"Setting read channel to {supported_eid}")
            func = state_dict[key]["block_relay"].set_lz_read_channel(supported_eid)

        # assign read lib to send and receive on read channel
        try:
            logging.info(
                f"Setting read lib (send direction) to {state_dict[key]['read_lib']} on {key}"
            )
            state_dict[key]["block_relay"].set_lz_send_lib(
                read_channel, state_dict[key]["read_lib"]
            )
        except Exception:
            pass
        try:
            logging.info(
                f"Setting read lib (receive direction) to {state_dict[key]['read_lib']} on {key}"
            )
            state_dict[key]["block_relay"].set_lz_receive_lib(
                read_channel, state_dict[key]["read_lib"]
            )
        except Exception:
            pass

### c) Set peers

In [ ]:
for read_key in state_dict.keys():
    # cycle through read-supporting chains
    if read_key == main_chain or state_dict[read_key]["read_lib"] == "unavailable":
        continue

    contract_read = state_dict[read_key]["block_relay"]

    read_eid = state_dict[read_key]["eid"]
    read_oapp = contract_read.address
    LZ_READ_CHANNEL = contract_read.LZ_READ_CHANNEL()
    # set self as read peer (so read requests can be sent)
    if contract_read.LZ_PEERS(LZ_READ_CHANNEL) != read_oapp:
        contract_read.set_peer(LZ_READ_CHANNEL, read_oapp)
        logging.info(f"Set peer on {read_key}")
    else:
        logging.info(f"Skipping {read_key} - already set")

    ## nested cycle through all chains
    for key in state_dict.keys():
        if key == main_chain or key == read_key:
            # skip main and current read chain
            continue

        contract_receiver = state_dict[key]["block_relay"]
        receiver_eid = state_dict[key]["eid"]
        receiver_oapp = contract_receiver.address
        # add read relayer as peer
        try:
            contract_receiver.set_peer(read_eid, read_oapp)
            logging.info(
                f"{key} - set peer {read_oapp} (eid {read_eid}) for messenger {contract_receiver.address}"
            )
        except Exception:
            logging.info(f"Skipping {key} - already set")
        # add chain to read broadcast targets
        try:
            contract_read.add_broadcast_target(
                state_dict[key]["eid"], state_dict[key]["block_relay"].address
            )
            logging.info(f"{read_key} - added {key} to broadcast targets")
        except Exception:
            logging.info(f"Skipping {read_key} - already set")
        # set chain to peers of read relayer
        if contract_read.LZ_PEERS(receiver_eid) != receiver_oapp:
            contract_read.set_peer(receiver_eid, receiver_oapp)
            logging.info(
                f"{read_key} - set peer {receiver_oapp} (eid {receiver_eid}) for messenger {contract_read.address}"
            )
        else:
            logging.info(f"Skipping {read_key} - already set")

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
def get_vyper_abi(filepath):
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


ABI_RELAY = get_vyper_abi("../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../contracts/BlockOracle.vy")
for key in state_dict.keys():
    if key == main_chain:
        continue
    state_dict[key]["block_relay_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["block_relay"].address, abi=ABI_RELAY
    )
    state_dict[key]["oracle_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["oracle"].address, abi=ABI_ORACLE
    )
account = Web3().eth.account.from_key(deployer.key)


def send_tx_single(w3, func, acc, value=0, gas_multiplier=2):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    tx["gas"] = int(gas_multiplier * w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value)
            success = True
        except Exception as e:
            if "replacement transaction underpriced" in str(e) or "nonce too low" in str(e):
                print(str(e), "Retrying...")
                success = False
                time.sleep(0.5)
            else:
                raise e
    return tx_hash

### Simple reads without broadcast

In [ ]:
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"Current block number on {main_chain}: {main_block}")
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        logging.info(f"Skipping {key}")
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]

    # First quote read fee
    fee = contract_w3.functions.quote_read_fee().call()
    logging.info(f"Read fee: {fee} on {key}")

    # Then request read
    func = contract_w3.functions.request_block_hash([], [])
    tx_hash = send_tx(state_dict[key]["w3"], func, account, 3 * fee)
    logging.info(f"Tx: {tx_hash.hex()} on {key}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    logging.info(f"Number difference: {main_block - number}")

### Now reads with broadcast

In [ ]:
# we can quote read fee now
# let's pick one of the read chains and quote fee
broadcaster = "base-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

BROADCAST_GAS = 2_000_000
read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
    0, BROADCAST_GAS, sum(broadcast_fees)
).call()
logging.info(f"LZRead fee with broadcast: {read_fee_with_broadcast} on {broadcaster}")

# magic broadcast call
main_block = state_dict[main_chain]["w3"].eth.block_number
func = broadcaster_w3.functions.request_block_hash(receive_eids, broadcast_fees, 0, BROADCAST_GAS)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, read_fee_with_broadcast)
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    logging.info(f"Number difference: {main_block - number}")